# Extracting functional traits (i.e. gene/metabolic pathway abundance profiles) from a MTG time series dataset (ALOHA station) using HUMANN3 plus a plankton-specific database.  

#### Samples have been described in the following paper: [Marine microbial metagenomes sampled across space and time](https://www.nature.com/articles/sdata2018176).


   
#### The code developed in this Notebook is meant for engineering our first molecular trait-based biodiversity asset from MTG datasets.  


In [ ]:
###NOTE:Uncomment below for pip install for outside requirements
# !pip3 install -r data/requirements_nb1.txt

In [ ]:
!$(which pip3) --version

In [1]:
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
import colorsys
from matplotlib.collections import PatchCollection

import Bio.SeqIO as bioseqio
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
#from Bio.Alphabet import IUPAC
from Bio import Entrez

from ete3 import NCBITaxa
from taxonomy_ranks import TaxonomyRanks

from subprocess import Popen, call, STDOUT, PIPE
import os
import shutil
import pandas as pd
import numpy as np
import matplotlib
import json
import glob
import re
import gzip
import sys
import csv
import time
import io
import pathlib
from collections import OrderedDict
import configparser

import pickle
import bz2 

from IPython.display import Image
from itertools import combinations
import itertools

In [2]:
#Importing rpy to call R functions within Python
import rpy2
import rpy2.robjects as robjects

#To install new packages we have to indicate the repo/CRAN
# utils.install_packages('ggplot2', repos='http://cran.us.r-project.org')
# utils.install_packages('mFD', repos='http://cran.us.r-project.org')

from rpy2.robjects.packages import importr, data
utils = importr('utils')
base = importr('base')
ggplot2 = importr('ggplot2')
# mFD = importr('mFD')
## To aid in printing HTML in notebooks
# import rpy2.ipython.html
# rpy2.ipython.html.init_printing()

In [3]:
pd.set_option('mode.chained_assignment', None)
matplotlib.rcParams['savefig.dpi'] = 1000
matplotlib.rcParams['figure.dpi'] = 1000
sns.set_style("whitegrid", {'axes.grid' : False})
sns.set_context("paper")
sns.set(font='serif')
sns.set_style('ticks')

In [4]:
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
###Uncomment below if necessary
rendef = "png" #"pdf" 
fig_renderer = pio.renderers[rendef]
fig_renderer.width = 1000
fig_renderer.height = 1000
pio.renderers.default = rendef

Utility functions

In [5]:
def format_tax_lbl(taxid = '94617'):
    """Function to properly format taxonomic labels using NCBI notation"""
    
    tax_lvls_lbls = ['species','genus','family', 'order', 'class', 'phylum', 'superkingdom'][::-1]
    rank_taxon = TaxonomyRanks(taxid)
    rank_taxon.get_lineage_taxids_and_taxanames()
    rank_dict =  list(rank_taxon.lineages.values())[0]
    tax_tree = list(rank_taxon.lineages.values())[0]
    
    #Parsing info on high rank tanks is optional, but can be quite handy when low ranks are unclassified: for higher rank assignment purposes if needed
    tax_ranks_list = [tax_tree[t][0].replace("NA","unclassified") for t in tax_lvls_lbls]
    ncbi_taxIDs_list = [str(tax_tree[t][1]) for t in tax_lvls_lbls]
    return ncbi_taxIDs_list,tax_ranks_list 

def parse_lineage(lineage_str):
    '''Function to correctly parse lineage using NCBI tax ID'''
    return (";".join(format_tax_lbl(lineage_str.split(';')[-1].replace('_', ' '))[-1])).replace(" ","_")

In [6]:
#Read config.ini file
config_obj = configparser.ConfigParser()

config_obj.read("data/configfile.ini")

user_info = config_obj["user_info"]
#Advisable to tell NCBI who you are
Entrez.email = user_info["user_email"]

humann3_profiling_results = config_obj["humann3_profiling_results"]
#Set folder where humann3 profiling results were stored
humann_output_folder = humann3_profiling_results["humann_output_folder"]
#Set the type of UniRef IDs used to conduct the profiling
uni_ref_rxn = humann3_profiling_results["uni_ref_rxn"]

#Indicate conda environment for the biobakery suite tool 
conda_env_biobakery_path = config_obj["biobakery3_bin"]["conda_env_biobakery_path"]

#Indicate bin dir for biobakery tools
biobakery3_bin = config_obj["biobakery3_bin"]["bin_dir"]

#Indicate where DIAMOND (protein alignment tool) is
dmnd_bin = config_obj["biobakery3_bin"]["dmnd_bin"]

#Indicate path to plankton-specific DB
plankton_specific_dmnd_db = config_obj["plankton_specific_db"]["dmnd_db"]

In [7]:
###Print info about the plankton-specific db used to conduct the functional profiling of the MTG data analyzed in this NB. Using Diamond
# !echo $plankton_specific_dmnd_db
# !$dmnd_bin dbinfo --db $plankton_specific_dmnd_db

In [8]:
###Use seqkit to pull out pattern-matching sequences from a multi-fasta file
###To install do: conda install -c bioconda seqkit
# !seqkit grep -r -n -p '.*Dinophyceae.*' $input_fasta >> $output_fasta

In [9]:
###List the wide range of functions shipped with humann3
# !ls -h `echo $biobakery3_bin"humann*"`

In [10]:
# !/dodrio/scratch/projects/starting_2022_048/envs/biobakery3/bin/humann3 --help

In [11]:
###Run bash script to join, renormalize, regroup and rename tables generated by humann. 
###Great utility functions shipped with humann are being invoked within this script
###NOTE: any anlysis performed below will depend on the output generated by this script!
# !bash ../../scripts/join_norm_humann_tables.sh $conda_env_biobakery_path $humann_output_folder $uni_ref_rxn

Metadata for the MTG samples can be found in the following file

In [12]:
#First load in SRR IDs for the MTG samples
ssr_ids = [os.path.basename(i).split("_")[0] for i in glob.glob(humann_output_folder + "/*pathabundance.tsv")]
# ssr_ids = pd.read_csv("../../data/reads/ALOHA_HOT_5m/ssr_ids.txt", header=None).values.flatten()
#Load in the whole set of samples
geotraces_hot_bats_metadata = pd.read_csv("../../data/reads/GEOTRACES_HOT_BATS.tsv",sep="\t")
hot_metadata = geotraces_hot_bats_metadata[geotraces_hot_bats_metadata["Cruise series"]=="HOT"]
hot_metadata.reset_index(drop=True, inplace=True)
print("Total number of samples processed by HUMANN: {}".format(len(ssr_ids)))
hot_metadata.head()

Total number of samples processed by HUMANN: 18


,Sample name,Cruise series,GEOTRACES section,Cruise ID,Cruise Station,Collection Date,Depth (m),env_biome,env_feature,env_material,...,Bottle ID,NCBI BioProject,NCBI SRA Study,NCBI BioSample,NCBI SRA Accession,Total read pairs,Total bases,NCBI SRA Accession for assembled metagenome contigs,Assembly method and version,Unnamed: 21
0,S0501,HOT,NaN,HOT156,Station ALOHA,2004-02-24,4,ocean_biome,ocean,water,...,1560200314,PRJNA385855,SRP109831,SAMN07136998,SRR5720299,13333617,4000085100,SRZ187079,metaSPAdes v3.9.0,NaN
1,S0502,HOT,NaN,HOT156,Station ALOHA,2004-02-24,"98,7",ocean_biome,ocean,water,...,1560200308,PRJNA385855,SRP109831,SAMN07136999,SRR5720300,10540927,3162278100,SRZ187080,metaSPAdes v3.9.0,NaN
2,S0503,HOT,NaN,HOT156,Station ALOHA,2004-02-24,"173,3",ocean_biome,ocean,water,...,1560200304,PRJNA385855,SRP109831,SAMN07137000,SRR5720301,12242781,3672834300,SRZ187081,metaSPAdes v3.9.0,NaN
3,S0504,HOT,NaN,HOT158,Station ALOHA,2004-04-20,"23,8",ocean_biome,ocean,water,...,1580200313,PRJNA385855,SRP109831,SAMN07137001,SRR5720302,41507610,12452283000,SRZ187082,metaSPAdes v3.10.1,NaN
4,S0505,HOT,NaN,HOT158,Station ALOHA,2004-04-20,"124,4",ocean_biome,ocean,water,...,1580200306,PRJNA385855,SRP109831,SAMN07137002,SRR5720303,26554323,7966296900,SRZ187083,metaSPAdes v3.9.0,NaN


In [13]:
#Here we extract the SRR_IDs we will focus the anlysis on
hot_time_series_metada = pd.concat([hot_metadata[hot_metadata["NCBI SRA Accession"]==i] for i in ssr_ids])
#Change dtype to datetime
hot_time_series_metada["Collection Date"] = pd.to_datetime(hot_time_series_metada["Collection Date"].apply(lambda s: s.replace("-","")), format='%Y%m%d')
#Samples sorted by collection time
hot_time_series_metada_sorted = hot_time_series_metada.sort_values(by=['Collection Date'])
#Dropping last row/sample due to date being too far in the future
# hot_time_series_metada_sorted = hot_time_series_metada_sorted.iloc[:-1,:].reset_index(drop=True)
#Drop time from datetime
# hot_time_series_metada_sorted["Collection Date"] = hot_time_series_metada_sorted["Collection Date"].dt.date
#Map SRR ID to date to have a way to plot time series
srrid2date_dict = hot_time_series_metada_sorted[["NCBI SRA Accession","Collection Date"]].set_index("NCBI SRA Accession").to_dict()["Collection Date"]
#Reformatting values to Week/year
srrid2date_dict2 = {k:"Week:{0} ({1})".format(v.weekofyear,v.year) for (k,v) in srrid2date_dict.items()}

### Pathway abundance analysis  
CPM = counts per million mapped reads

In [14]:
#Loading data table
cpm_path_abund_df = pd.read_csv(os.path.join(humann_output_folder,'joint_samples_path_abundance-cpm.tsv'),sep='\t')
cpm_path_abund_df.set_index('# Pathway',inplace=True)
cpm_path_abund_df.index.name = 'PathwayID'

# Filtering observations/pathways that have been properly identified at the species level
filtered_cpm_path_abund_df = cpm_path_abund_df.loc[~cpm_path_abund_df.index.str.contains('UNMAPPED|UNINTEGRATED', regex=True)]
#DF with properly stratified counts across species
filtered_cpm_path_abund_sps_strat_df = filtered_cpm_path_abund_df.loc[filtered_cpm_path_abund_df.index.str.contains('g__')]
#Rename columns
filtered_cpm_path_abund_sps_strat_df.columns = filtered_cpm_path_abund_sps_strat_df.columns.map(lambda s: s.split('_')[0])
filtered_cpm_path_abund_sps_strat_df = filtered_cpm_path_abund_sps_strat_df[hot_time_series_metada_sorted["NCBI SRA Accession"]]
#Change columns to time in weeks/year
filtered_cpm_path_abund_sps_strat_df.columns = filtered_cpm_path_abund_sps_strat_df.columns.map(srrid2date_dict2)

#DF with pathway counts/abundance aggregated at the community level
filtered_cpm_path_abund_community_wide_df = filtered_cpm_path_abund_df.loc[~filtered_cpm_path_abund_df.index.str.contains('g__')]
#Rename columns
filtered_cpm_path_abund_community_wide_df.columns = filtered_cpm_path_abund_community_wide_df.columns.map(lambda s: s.split('_')[0])
filtered_cpm_path_abund_community_wide_df = filtered_cpm_path_abund_community_wide_df[hot_time_series_metada_sorted["NCBI SRA Accession"]]
#Change columns to time in weeks/year
filtered_cpm_path_abund_community_wide_df.columns = filtered_cpm_path_abund_community_wide_df.columns.map(srrid2date_dict2)

### Visualizing community aggregated metabolic pathway abundances through time

In [15]:
#Set threshold pathway abundnace value for better visualization
#Sorting index by grand total across samples
sorted_idx = filtered_cpm_path_abund_community_wide_df.sum(axis=1).sort_values(ascending=False)
#Set a threshold value to shrink matrix size
commun_aggregate_pwy_abund_thr = np.quantile(sorted_idx,0.25)
sorted_pwys_by_grand_total = sorted_idx[sorted_idx >= commun_aggregate_pwy_abund_thr].index
#Sorted by index
filtered_cpm_path_abund_community_wide_sorted_df = filtered_cpm_path_abund_community_wide_df.loc[sorted_pwys_by_grand_total]
#Take the log10 transformed with pseudo-count
filtered_cpm_path_abund_community_wide_sorted_log_trans_df = filtered_cpm_path_abund_community_wide_sorted_df.applymap(lambda x: np.log10(1 + x))
#Simplify pwy annotation
filtered_cpm_path_abund_community_wide_sorted_log_trans_df.index = filtered_cpm_path_abund_community_wide_sorted_log_trans_df.index.map(lambda s: s.split("|")[0])

In [16]:
#Peeking at the df
filtered_cpm_path_abund_community_wide_sorted_log_trans_df.T.head()

PathwayID,PWY-3781: aerobic respiration I (cytochrome c),VALSYN-PWY: L-valine biosynthesis,SER-GLYSYN-PWY: superpathway of L-serine and glycine biosynthesis I,PWY-1042: glycolysis IV,PWY-7229: superpathway of adenosine nucleotides de novo biosynthesis I,PWY-7279: aerobic respiration II (cytochrome c) (yeast),PWY-8178: pentose phosphate pathway (non-oxidative branch) II,PWY-6126: superpathway of adenosine nucleotides de novo biosynthesis II,PWY-7220: adenosine deoxyribonucleotides de novo biosynthesis II,PWY-7222: guanosine deoxyribonucleotides de novo biosynthesis II,...,PWY-622: starch biosynthesis,P221-PWY: octane oxidation,PWY-3781: aerobic respiration I (cytochrome c),SER-GLYSYN-PWY: superpathway of L-serine and glycine biosynthesis I,PWY-8011: L-serine biosynthesis II,PWY-5347: superpathway of L-methionine biosynthesis (transsulfuration),METSYN-PWY: superpathway of L-homoserine and L-methionine biosynthesis,MET-SAM-PWY: superpathway of S-adenosyl-L-methionine biosynthesis,P42-PWY: incomplete reductive TCA cycle,PWY-5136: fatty acid &beta;-oxidation II (plant peroxisome)
Week:3 (2003),2.961215,2.882412,2.502441,2.534694,2.459691,2.438699,2.452921,2.376991,2.233623,2.233623,...,0.0,0.000000,0.765686,0.000000,0.466712,0.0,0.0,0.0,0.000000,0.000000
Week:9 (2003),3.041957,2.966225,2.607031,2.599281,2.525493,2.578405,2.501259,2.455495,2.341903,2.341903,...,0.0,0.000000,0.000000,0.495553,0.232252,0.0,0.0,0.0,0.000000,0.000000
Week:13 (2003),3.002900,2.881578,2.549508,2.553090,2.464737,2.530438,2.592844,2.382421,2.236777,2.236777,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.308402,0.000000
Week:17 (2003),2.981724,2.821072,2.546890,2.490314,2.435784,2.507357,2.292074,2.379596,2.304770,2.304770,...,0.0,0.800193,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.231997
Week:21 (2003),2.984144,2.874648,2.617747,2.540530,2.530618,2.477447,2.371349,2.490280,2.459777,2.459777,...,0.0,0.000000,0.000000,0.219629,0.356729,0.0,0.0,0.0,0.000000,0.000000


In [18]:
### To display the whole clustermap uncomment below

# g = sns.clustermap(filtered_cpm_path_abund_community_wide_sorted_log_trans_df,
#                    method='ward',
#                    metric='euclidean',
#                    col_cluster=False,
#                    vmin=filtered_cpm_path_abund_community_wide_sorted_log_trans_df.values.ravel().min(), 
#                    vmax=filtered_cpm_path_abund_community_wide_sorted_log_trans_df.values.ravel().max(), 
#                    cmap=plt.cm.Reds, 
#                    yticklabels=True, 
#                    linewidth=1, 
#                    linecolor="white",
#                    figsize=(40,30),
#                    dendrogram_ratio=0.15,
#                    cbar_pos=None
# #                    cbar_kws=dict(ticks=[0, 0.50, 1], orientation='ver')
#                   )

# # g.cax.set_visible(False)
# _ = g.ax_heatmap.set_yticklabels(g.ax_heatmap.get_yticklabels(), fontsize=9)
# plt.savefig('clustermap.png', dpi=300, bbox_inches='tight')
# plt.close(g.fig)

In [ ]:
###To display the clustermap above run code below
# from IPython.display import Image, display
# display(Image(filename='clustermap.png'))

### Let's focus now on stratified analysis: creating a matrix with species as rows and PWYs as columns.  
This is the type of matrix required to compute multidimensional functional diversity space with the tool [mFD](https://cmlmagneville.github.io/mFD/index.html), as described in the paper [mFD: an R package to compute and illustrate the multiple facets of functional diversity](https://onlinelibrary.wiley.com/doi/10.1111/ecog.05904)

In [18]:
# #Get stratified data: taxon-specific metabolic pathways
# sampling_points = filtered_cpm_path_abund_sps_strat_df.columns

# #Then to keep samples organized identically when looking at them in terms of contributing species to particular pwys, let's use as ref the pwys list from community wide traits
# core_pwys_list = []
# for ix in filtered_cpm_path_abund_community_wide_sorted_log_trans_df.index:
#     pwy_in = [n for n in filtered_cpm_path_abund_sps_strat_df.index if ix in n]
#     if len(pwy_in)>0:
#         core_pwys_list.append(ix)

In [34]:
# ###Create a dict mapping sampling event to stratified metabolic pathway profiles
# event2pwyprofile = {}

# #Loop over the set of sampling events (time points)
# for sp in sampling_points:

#     #Filtering df
#     bool_sps = filtered_cpm_path_abund_sps_strat_df[sp].index.str.contains("|g__")
#     strat_indexes = filtered_cpm_path_abund_sps_strat_df[sp][bool_sps].index.values
#     #Sort pwys by total abundance across samples
#     sorted_strat_by_total_sum = filtered_cpm_path_abund_sps_strat_df.loc[strat_indexes].sum(axis=1).sort_values(ascending=False)
#     #Set a threshold value to shrink matrix size
#     # strat_pwy_abund_thr = np.quantile(sorted_strat_by_total_sum,0.25)
#     #Filter all samples based on predefined cutoff
#     # filtered_cpm_path_abund_sps_strat_df2 = filtered_cpm_path_abund_sps_strat_df.loc[sorted_strat_by_total_sum[sorted_strat_by_total_sum>=strat_pwy_abund_thr].index]
#     filtered_cpm_path_abund_sps_strat_df2 = filtered_cpm_path_abund_sps_strat_df.loc[sorted_strat_by_total_sum.index]

#     #Sanity check to keep pathway annotations stratified at genus/species level
#     pwy_annot = filtered_cpm_path_abund_sps_strat_df2.index.map(lambda s: s.split("|")[0])
#     lineage_annot = filtered_cpm_path_abund_sps_strat_df2.index.map(lambda s: s.split("|")[-1].replace(";g__", " FX;g__").split(" FX;")[-1])
#     #Clean index names
#     reformat_idx_names = ["|".join(s) for s in zip(pwy_annot,lineage_annot)]
#     filtered_cpm_path_abund_sps_strat_df2.index = reformat_idx_names

#     #Create a dict with SRR_IDs as keys and stratified matrix of pyw abundances as values
#     # for sp in sampling_points:
#     pwy2sps_list = filtered_cpm_path_abund_sps_strat_df2[sp].index.map(lambda s: s.split("|"))
#     #Recast data as a df
#     new_list = []
#     for (l,i) in zip(pwy2sps_list,filtered_cpm_path_abund_sps_strat_df2[sp].values):
#         l.append(i)
#         new_list.append(l)

#     #Cast data into a df
#     strat_df = pd.DataFrame(new_list, columns=["pwy_annot","species","pwy_abund"])
#     #Create a pivot table to reshape the df all at once
#     strat_pivot_table = strat_df.pivot_table(index='species', columns='pwy_annot', values='pwy_abund', aggfunc='sum').fillna(0)
#     #Put the df into the right format for visualization via a heatmap
#     strat_log_transf_df = strat_pivot_table.applymap(lambda x: np.log10(1+x))
    
#     event2pwyprofile[sp] = strat_log_transf_df
    
# ###Using pickle to dump the dict generated above (serialize)
# with open('data/strat_pwy_profiles_per_sample.pickle', 'wb') as handle:
#     pickle.dump(event2pwyprofile, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Loading dict to create some plots and run some analysis

In [46]:
# Load data (deserialize): a dict mapping sampling events to stratified metabolic pathway activities 
with open('data/strat_pwy_profiles_per_sample.pickle', 'rb') as handle:
    event2pwyprofile = pickle.load(handle)

### Visualizing stratified contributions per species for one single sampling point/MTG: clustermap

In [48]:
# strat_log_transf_df = event2pwyprofile['Week:3 (2003)']
# strat_traits_data = strat_log_transf_df.loc[strat_log_transf_df.sum(axis=1).replace(0, np.nan).dropna().index,
#                             strat_log_transf_df.sum().replace(0, np.nan).dropna().index]

In [21]:
### To run a clustermap for a stratified sample, uncomment code below
# g = sns.clustermap(strat_traits_data,
#                    method='ward',
#                    metric='euclidean',
#                    col_cluster=False,
#                    vmin=strat_traits_data.values.ravel().min(), 
#                    vmax=strat_traits_data.values.ravel().max(), 
#                    cmap=plt.cm.Reds, 
#                    yticklabels=True, 
#                    linewidth=1, 
#                    linecolor="white",
#                    figsize=(40,30),
#                    dendrogram_ratio=0.15,
#                    cbar_pos=None
# #                    cbar_kws=dict(ticks=[0, 0.50, 1], orientation='ver')
#                   )

# # g.cax.set_visible(False)
# _ = g.ax_heatmap.set_yticklabels(g.ax_heatmap.get_yticklabels(), fontsize=9)
# plt.savefig('clustermap_strat_sps.png', dpi=300, bbox_inches='tight')
# plt.close(g.fig)

In [49]:
### Alternatively, just load in figure from file
from IPython.display import Image, display
display(Image(filename='clustermap_strat_sps.png'))

### Applying a diffusion map to observe distribution of data points in lower dimensional space

In [56]:
for k in event2pwyprofile.keys():
    flatten_data = event2pwyprofile[k].values.ravel()
    print(flatten_data.shape[0])

11715
11715
11715
11715
11715
11715
11715
11715
11715
11715
11715
11715
11715
11715
11715
11715
11715
11715


In [63]:
key_list = []
val_list = []
for k,v in event2pwyprofile.items():
    key_list.append(k)
    val_list.append(v.values.ravel())

In [67]:
pwy_feats_df = pd.DataFrame(val_list).T
pwy_feats_df.columns = key_list

In [68]:
pwy_feats_df

,Week:3 (2003),Week:9 (2003),Week:13 (2003),Week:17 (2003),Week:21 (2003),Week:25 (2003),Week:29 (2003),Week:34 (2003),Week:45 (2003),Week:51 (2003),Week:9 (2004),Week:21 (2004),Week:25 (2004),Week:33 (2004),Week:40 (2004),Week:44 (2004),Week:48 (2004),Week:52 (2004)
0,0.0,0.0,0.0,0.0,0.0,0.027968,0.047699,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003809,0.031802,0.0
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11710,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
11711,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
11712,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
11713,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0
